In [357]:
import sys
sys.path.append("..")
import sound_eval as sev
import param_generation as pg
from common_vars import *
import imp
imp.reload(pg)
imp.reload(sev)
import imp
from deap import base, creator
import random
from deap import tools
import scipy.spatial.distance as d
from helpers import rToParams
from IPython.display import Audio


In [517]:
toolbox = base.Toolbox()
STACK_SIZE = 4
POP_SIZE = 10
GREP,MUTPB = 0.5, 1
MUTENV,MUTTEXT=0.3,0.2

class drum_program():
    def __init__(self,stack_num=STACK_SIZE):
        self.program=[pg.RandomParams() for i in range(stack_num)]
    def mutate(self):
        for p in self.program:p.mutate(e=0.8,t=0.8,s=0.2)
        return self
creator.create("FitnessMin", base.Fitness, weights=(1,))
creator.create("Individual",drum_program, fitness=creator.FitnessMin)

drum="kick"

def initPopulation(pcls,ng):
    ipop=[creator.Individual()
          for x in range(ng)]
    return pcls(ipop)

def evaluate(individual):
    o=pg.ensemble(individual.program)
    return (sev.evalSound(o),)

def randSelect(l,k):
    return random.sample(l, k)

# toolbox.register("individual_guess", initIndividual, creator.Individual)
toolbox.register("population_guess", initPopulation, list,)


toolbox.register("select_best", tools.selBest,fit_attr='fitness')
toolbox.register("select_random",randSelect)
toolbox.register("evaluate", evaluate)

hof=tools.HallOfFame(POP_SIZE*2)

In [ ]:
%%time


def evolve(pop,hof):
    newGenes=toolbox.population_guess(int(0.25*POP_SIZE))
    offspring = pop.copy() + newGenes

    # Apply crossover and mutation on the offspring
#     for j,child in enumerate(offspring):
#         for i,program in enumerate(child.program):
#             if random.random() < GREP:                
#                 child.program[i]=random.choice(hof[0:3]).program[i]
#                 del child.fitness.values
#         offspring[j]=child
#         pass

    # Clone the selected individuals        
    for i,mutant in enumerate(offspring):
        if random.random() < MUTPB:
            offspring[i]=mutant.mutate()
            del offspring[i].fitness.values

    # Evaluate the individuals with an invalid fitness
    invalid_ind = [ind for ind in offspring if not ind.fitness.valid]
    fitnesses = map(toolbox.evaluate, invalid_ind)
    for ind, fit in zip(invalid_ind, fitnesses):
        ind.fitness.values = fit
    #update hof

    survivors = toolbox.select_best(offspring,POP_SIZE)
    return survivors.copy(),hof
    
NGEN=50
def main(hof):
    pop = toolbox.population_guess(POP_SIZE)
    fitnesses = map(toolbox.evaluate, pop)
    for ind, fit in zip(pop, fitnesses):
        ind.fitness.values = fit
    for g in range(NGEN):
        pop,hof=evolve(pop,hof)
        hof.update(pop)
        print (g,"\r",end="")
        for hofer in hof[0:5]:
            print(g,hofer.fitness,evaluate(hofer))
    return pop
pop=main(hof)
# print([i.fitness.values for i in pop])

0 (9.45176554210947e-13,) (7.498441971509617e-13,)
0 (1.7769294336725944e-14,) (5.079216127551467e-14,)
0 (4.9351216618914235e-17,) (2.7231050463041688e-17,)
0 (2.5847706693832507e-18,) (3.0668695215574782e-18,)
0 (2.4496401697699035e-18,) (3.7574923343501985e-18,)
1 (6.444865752186502e-10,) (6.425778797947146e-10,)
1 (9.45176554210947e-13,) (1.0088620477216592e-12,)
1 (1.7769294336725944e-14,) (7.850655076111629e-14,)
1 (1.960033710550859e-16,) (2.3138139616305626e-16,)
1 (4.9351216618914235e-17,) (2.5703477827866536e-17,)
2 (6.444865752186502e-10,) (6.266590579784292e-10,)
2 (9.45176554210947e-13,) (1.006365563799294e-12,)
2 (1.7769294336725944e-14,) (6.40656486737036e-14,)
2 (3.3707427320056626e-16,) (3.538847272997915e-16,)
2 (1.960033710550859e-16,) (2.2564584490962177e-16,)
3 (6.444865752186502e-10,) (7.231825138731551e-10,)
3 (9.45176554210947e-13,) (7.986770966805778e-13,)
3 (1.7769294336725944e-14,) (1.1073859385903051e-13,)
3 (7.623609927479187e-15,) (2.8779214932385735e-14,)

In [496]:
k=hof[2]
print(k.program,"prog")

o=pg.ensemble(k.program)
print(evaluate(k),k.fitness.values)

Audio(o,rate=SR,autoplay=True,embed=True)

[<param_generation.RandomParams object at 0x7f9f0dd98310>, <param_generation.RandomParams object at 0x7f9f0de71f90>] prog
(1.6128220287070505e-11,) (0.8676271438598633,)


In [516]:

from ipywidgets import GridspecLayout,HBox,widgets
# What did our DAEP search return?
z=hof
x,y=3,int((len(z)/4))
grid=[]
grid = GridspecLayout(x, y)
for i in range(x):
    for j in range(y):
        o= widgets.Output()
        sample_number=i+j+(i*(y-1))
        pset=z[sample_number]
        try:
            out=pg.ensemble(pset.program)
            with o:
                display(Audio(out,rate=SR,autoplay=False,embed=True),pset.fitness.values,evaluate(pset))
            grid[i, j] = o
        except:
            continue
display(grid)

(0.8762537240982056,)

(0.8762537240982056,)

(0.8658111691474915,)

(0.8665728569030762,)

(0.8614299893379211,)

...

(0.037834491580724716,)

(0.03030778281390667,)

(0.029421987012028694,)

(0.029421987012028694,)

GridspecLayout(children=(Output(layout=Layout(grid_area='widget001')), Output(layout=Layout(grid_area='widget0…

In [356]:
# z=toolbox.population_guess(2)[1]

# print(pg.ensemble(z.program))
# print(z)
# print(evaluate(z))
# for x in z.program:
#     print(x.__dict__)
#     for i in range(10):
#         x=x.mutate()
#     print(x.__dict__)

print(evaluate(z))
Audio(pg.ensemble(z),rate=SR,autoplay=True)

AttributeError: 'list' object has no attribute 'program'